In [ ]:
# shared state object

class Nest:
  def __init__(self):
    self.input_features = [0.5, 0.5, 0.5] # tied to resources maybe? emergency situations? which impacts ant day to day?

    # do i want to convey resources in terms of need or amount? how does a worker ant change resource values + input features?
    self.resources = {
        'food': 100,
        'babies': 20 # TODO -- HOW ARE MORE BABIES MADE? SUCCESS OF NURSES? OR A CERTAIN NEST STATE FOOD WISE
    }
    self.danger = 3.0

    # TODO - NORMALIZE PHEROMONE VALUES: if one is out of 1 and other is out of 10...
    # role counts
    self.nurses = 0
    self.foragers = 0
    self.fighters = 0
    self.dead = 0

    self.n_scarcity = 0
    self.f_scarcity = 0

  def nursing_scarcity(self):
    if self.resources['babies'] == 0:
        return 0.0
    coverage_ratio = (3 * self.nurses) / self.resources['babies']
    scarcity = max(0.0, (1 - coverage_ratio) * 100)
    return scarcity

  def food_scarcity(self):
    population = self.nurses + self.foragers + self.fighters

    food_need = population * 2 # each ant eats 2

    #food_scarcity = max(0.0, (food_need - self.resources['food']) / food_need) # **TODO - cap between 0 and 1

    return min(1.0, max(0.0, (food_need - self.resources['food']) / food_need))

  def tick(self):

    # Decay
    self.input_features[1] = max(0.0, self.input_features[1] * 0.98)
    self.input_features[2] = max(0.0, self.input_features[2] * 0.97)

    # Baby pheromone
    baby_signal = 0.02 * self.resources['babies']
    nurse_reduction = 0.03 * self.nurses
    self.input_features[0] = max(0.0, self.input_features[0] + baby_signal - nurse_reduction)

    # Scarcity updates
    n_scarcity = self.nursing_scarcity() # TODO - how to update this per tick, how are new ants initialized? with scarcity not updated but pheromones updated?
    f_scarcity = self.food_scarcity()


  def summary(self):
    print("\n--- Pheromones ---")
    print(f"Baby     : {self.input_features[0]:.2f}")
    print(f"Food     : {self.input_features[1]:.2f}")
    print(f"Danger   : {self.input_features[2]:.2f}")

    print("\n--- Scarcity ---")
    print(f"Baby     : {self.n_scarcity:.2f}")
    print(f"Food     : {self.f_scarcity:.2f}")
    print(f"Danger   : {self.danger:.2f}")

    print("\n--- Resources ---")
    print(f"Food     : {self.resources['food']}")
    print(f"Babies   : {self.resources['babies']}")

  def concise_report(self, tick_report):
    print("\n--- Ant Status ---")
    print("Role      | Count | New | Action Summary")
    print("--------------------------------------------")

    print(f"Nurse     | {self.nurses:5} | {tick_report['nurse']['new']:3} | cared for babies")

    f = tick_report["forager"]
    print(f"Forager   | {self.foragers:5} | {f['new']:3} | {f['success']} success, {f['fail']} fail")

    f = tick_report["fighter"]
    print(f"Fighter   | {self.fighters:5} | {f['new']:3} | {f['win']} win, {f['retreat']} retreat, {f['death']} death")

    print(f"Dead      | {self.dead:5} | {tick_report['dead']['new']:3} | --")


In [ ]:
import numpy as np
import random

class Ant:

  # input_features and weights have to be [] lists
  def __init__(self, nest: Nest):
    self.nest = nest
    self.summary = "Ant Summary: \n------------\n" # append latest messages to this:
    self.role = ""
    self.role = self.decide_role()
    self.location = 0 # dummy for now
    self.perception = [self.nest.input_features] # update with scarcity and other stuff

    # TODO: SET UP EXECUTION ONCE ROLE HAS BEEN RECEIVED
  # makes decision for ant
  def decide_role(self):

    input_features = self.nest.input_features

    options = [
      [.3, .3, .4],
      [.3, .4, .3],
      [.4, .3, .3]
    ]

    weights = random.choice(options)

    x = np.array(input_features)
    theta = np.array(weights)

    z = theta*x

    exp_z = np.exp(z - np.max(z))
    softmax = exp_z/exp_z.sum()

    decision_index = np.argmax(softmax)

    role = ["nurse", "forager", "fighter"][decision_index]
    # Update role

    self.update_role(role)

    return role

  def update_role(self, new_role, tick_report=None):

    if self.role:
      if self.role == "nurse": self.nest.nurses -= 1
      elif self.role == "forager": self.nest.foragers -= 1
      elif self.role == "fighter": self.nest.fighters -= 1

    if tick_report and new_role != self.role: # TODO - understand this
        tick_report[new_role]["new"] += 1

    self.role = new_role
    if self.role == "nurse": self.nest.nurses += 1
    elif self.role == "forager": self.nest.foragers += 1
    elif self.role == "fighter": self.nest.fighters += 1



  def act(self, tick_report):

    if self.role == "dead":
        return

    self.summary += f"\nAnt Role: {self.role}"

    if self.role == "nurse":
      # babies naturally increase
      self.nest.input_features[0] = max(0.0, self.nest.input_features[0] - .1) # reduce baby pheromones
    elif self.role == "forager":
      if random.random() < 0.7: # if success
        self.nest.resources['food'] += 5
        self.nest.input_features[1] += .1 # increase food pheromones
        tick_report["forager"]["success"] += 1
      else:
        tick_report["forager"]["fail"] += 1


    elif self.role == "fighter":
      r = random.random()
      if r < 0.5: # success
        self.nest.danger = max(0.0, self.nest.danger - .2)
        self.nest.input_features[2] = max(0.0, self.nest.input_features[2] - .1) # reduce danger pheromones
        tick_report["fighter"]["win"] += 1
      elif r < 0.9:  # retreat - TODO: is fight pheromone increased? location and spatial, when to totally give up
        self.nest.input_features[2] += .1 # TODO - why no max function for this?
        tick_report["fighter"]["retreat"] += 1
      else: # death
        self.update_role("dead", tick_report)
        self.nest.danger += .5
        self.nest.input_features[2] += .2
        tick_report["fighter"]["death"] += 1
        tick_report["dead"]["new"] += 1

    return self.summary.strip()

  # TODO: IMPLEMENT ROLE CHANGES
  def step(self):
    # update perception maybe
    # call decision()
    # choose new role
    # execute

    return

In [ ]:
nest = Nest()
ants = [Ant(nest) for _ in range(20)]

for step in range(100):
  print(f"\n=== TICK {step} ===")
  # Initialize tick report
  tick_report = {
      "nurse": {"new": 0},
      "forager": {"new": 0, "success": 0, "fail": 0},
      "fighter": {"new": 0, "win": 0, "retreat": 0, "death": 0},
      "dead": {"new": 0}
  }
  for ant in ants:
    ant.act(tick_report)
  nest.tick()

  nest.concise_report(tick_report)
  nest.summary()


# improve pheromone updating(normalize somehow to keep 1 as total or something) + baby pheromone became negative. add babies or something, or set that cap
# fix scarcity not being calculated
# how to keep adding ants(babies growing, i spawn em in, etc...)
# when ant die in fight, danger level increases while danger pheromone decreases. decay?


=== TICK 0 ===

--- Ant Status ---
Role      | Count | New | Action Summary
--------------------------------------------
Nurse     |     4 |   0 | cared for babies
Forager   |     9 |   0 | 2 success, 7 fail
Fighter   |     7 |   0 | 2 win, 5 retreat, 0 death
Dead      |     0 |   0 | --

--- Pheromones ---
Baby     : 0.38
Food     : 0.69
Danger   : 0.78

--- Scarcity ---
Baby     : 0.00
Food     : 0.00
Danger   : 2.60

--- Resources ---
Food     : 110
Babies   : 20

=== TICK 1 ===

--- Ant Status ---
Role      | Count | New | Action Summary
--------------------------------------------
Nurse     |     4 |   0 | cared for babies
Forager   |     9 |   0 | 8 success, 1 fail
Fighter   |     6 |   0 | 5 win, 1 retreat, 1 death
Dead      |     0 |   2 | --

--- Pheromones ---
Baby     : 0.28
Food     : 1.46
Danger   : 0.56

--- Scarcity ---
Baby     : 0.00
Food     : 0.00
Danger   : 2.10

--- Resources ---
Food     : 150
Babies   : 20

=== TICK 2 ===

--- Ant Status ---
Role      | Count | 